In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import nltk
import math
import numpy as np
import warnings
import mlflow
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

warnings.filterwarnings("ignore")

---
# 1. Extraindo trechos anotados dos XMLs

### 1.1. Adiquirindo a raiz de cada XML

In [ ]:
glob_path = 'xml_batch1/*.xml' # Caminho até os XMLs
roots = []
for xml in glob.glob(glob_path):
    tree = ET.parse(xml)
    roots.append(tree.getroot())

In [ ]:
# Esse bloco deve demorar de alguns segundos a 1 minuto
atos_csv_dict = {}

for root in roots: # Intera sobre as raizes
    for relation in root.findall(".//relation"):                            # Intera sobre as relações.
        row_act = {}
        type_relation = relation.find('.//infon[@key="type"]').text
        for node in relation.findall('node'):                               # Intera sobre os nós(anotações) da relação.
            ref_id = node.get('refid')
            annotation = root.find(f'.//annotation[@id="{ref_id}"]')        # Encontra anotação.
            type_annotation = annotation.find('.//infon[@key="type"]').text # Encontra tipo da anotação.
            text_annotation = annotation.find('text').text                  # Encontra texto da anotação.
            row_act[type_annotation] = text_annotation
        
        if type_relation not in atos_csv_dict:                              # Checa se a tabela já existe, caso contrário, cria uma.
            atos_csv_dict[type_relation] = pd.DataFrame()
            
        atos_csv_dict[type_relation] = atos_csv_dict[type_relation].append(row_act, ignore_index=True)

In [ ]:
nomeacao_df = atos_csv_dict['Ato_Nomeacao_Comissionado']

nomeacao_fields = ['nome', 'cargo_efetivo', 'matricula', 'matricula_SIAPE', 'simbolo', 'cargo_comissionado', 'hierarquia_lotacao', 'orgao', 'Ato_Nomeacao_Comissionado']

# Nessa linha todas as colunas que não pertencem a nomecao são extraidas para uma segunda lista. (Compreensão de listas).
nomeacao_non_fields = [column for column in nomeacao_df.columns if column not in nomeacao_fields]

# Exclusão de todas as linhas que possuam algum valor nos campos que não pertecem a nomeacao.
nomeacao_df = nomeacao_df[nomeacao_df[nomeacao_non_fields].isna().any(axis=1)]

# Exclusão de todas as colunas que não pertecem a nomeação.
nomeacao_df = nomeacao_df.drop(columns=nomeacao_non_fields)

# Exclusão das linhas que não possuem anotação de atos.
nomeacao_df = nomeacao_df.dropna(subset=['Ato_Nomeacao_Comissionado'])

atos_csv_dict['Ato_Nomeacao_Comissionado'] = nomeacao_df
atos_csv_dict['Ato_Nomeacao_Comissionado']

In [ ]:
for key in atos_csv_dict.keys():
    atos_csv_dict[key] = atos_csv_dict[key].dropna(subset=[key])

In [ ]:
_tokenizer = nltk.RegexpTokenizer(r"\w+")
def tokenize(sentence):
    try:
        new_words = _tokenizer.tokenize(sentence)
        return new_words    
    except:
        print("SENTENCE:", sentence)
        input()
    return ''


def find_entity(row, token):
    """ ...
    
    Assumes `row` has the whole text on the first collumn
    and the remaining contain entities.
    """
    for column in row.keys()[1:]:
        if (row[column] is not np.nan 
            and token == tokenize(row[column])[0]):
            return column
    return None

# Atualizar no futuro para qualquer ato.
# Aparentemente esse algoritmo está O(n*m) onde n é a quantidade de tokens e m a quantidade de colunas do df.
def generate_IOB_labels(row):
    """Generates IOB-labeling for whole text and entities.

    Assumes `row` has the whole text on the first collumn
    and the remaining contain entities.
    """
    labels = []
    entity_started = False
    text = row.iloc[0]
    for token in tokenize(text):                         # Intera sobre cada token da anotação do ato.
        if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
            entity = find_entity(row, token)                 # Busca o token atual no primeiro token de todos os campos do df.
            if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                entity_started = True
                token_index = 1
                labels.append('B-' + entity)
            else:
                labels.append('O')
        else:                                                # Caso uma entidade já tenha sido identificada
            if token_index < len(tokenize(row[entity])) and token == tokenize(row[entity])[token_index]: # Checa se o próximo token pertence à entidade e se o tamanho da entidade chegou ao fim.
                labels.append('I-' + entity)                 # Se a entidade ainda possui tokens e a comparação foi bem sucedida adicione o label I.
                token_index += 1
                if token_index >= len(tokenize(row[entity])):
                    entity_started = False
            else:                                            # Se o token n for igual ou a entidade chegou ao fim.
                entity_started = False
                labels.append('O')
                
    return labels


def find_entity(row, token, ignore_idx=0):
    """Searches for named entities on columns, except by ignore_idx-columns.
    
    ignore_idx: int indicating which column has
                the TEXT where the named were extracted from
    """
    for idx, column in enumerate(row.keys()):
        if idx == ignore_idx:
            continue
        if row[column] is not np.nan and token == tokenize(row[column])[0]:
            return column
    
    return None

# Atualizar no futuro para qualquer ato.
# Complexidade: O(n*m)
# n: quantidade de tokens
# m: quantidade de colunas do df.
def generate_IOB_labels(row, idx=0):
    """Generate IOB-labels for idx-column."""
    labels = []
    entity_started = False
    text = row.iloc[idx]
    for token in tokenize(text):                         # Intera sobre cada token da anotação do ato.
        if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
            entity = find_entity(row, token)                 # Busca o token atual no primeiro token de todos os campos do df.
            if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                entity_started = True
                token_index = 1
                labels.append('B-' + entity)
            else:
                labels.append('O')
        else:                                                # Caso uma entidade já tenha sido identificada
            if token_index < len(tokenize(row[entity])) and token == tokenize(row[entity])[token_index]: # Checa se o próximo token pertence à entidade e se o tamanho da entidade chegou ao fim.
                labels.append('I-' + entity)                 # Se a entidade ainda possui tokens e a comparação foi bem sucedida adicione o label I.
                token_index += 1
                if token_index >= len(tokenize(row[entity])):
                    entity_started = False
            else:                                            # Se o token n for igual ou a entidade chegou ao fim.
                entity_started = False
                labels.append('O')
                
    return labels


### 3.2. Agora, com as funções de geração label prontas iremos criar uma lista de strings representando os labels para adicionar ao df de cada ato.

---
# 4. Criação das features e treinamento do CRF à moda José

In [ ]:
# %%time
def extract_features(sentence):
    sentence_features = []
    for j, sent in enumerate(sentence):
        word_feat = {
                'word': sent.lower(),
                'capital_letter': sent[0].isupper(),
                'all_capital': sent.isupper(),
                'isdigit': sent.isdigit(),
                'word_before': sent.lower() if j==0 else sentence[j-1].lower(),
                'word_after:': sent.lower() if j+1>=len(sentence) else sentence[j+1].lower(),
                'BOS': j==0,
                'EOS': j==len(sentence)-1
        }
        sentence_features.append(word_feat)
    return sentence_features


def extract_rows_features(arq, idx=0):
    """
    Tokenizes then extract features from idx-column.
    """
    return arq.iloc[:, idx].map(tokenize).map(extract_features)



class IOB_Transformer:
    def __init__(self, idx=''):
#         self.idx = idx
        pass
    def fit(self, X=None, y=None, **fit_params):
        return self
    def transform(self, df):
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(' '.join(generate_IOB_labels(row)))
            except Exception as e:
                print(row)
                raise e

        return pd.Series(labels_row).str.split()


    
class FeatureTransformer:
    def __init__(self, key=''):
        self.key = key
    def fit(self, X=None, y=None, **fit_params):
        return self
    def transform(self, df):        
        return extract_rows_features(df)


In [ ]:
atos_csv_dict['Ato_Cessao'].columns

In [ ]:
ss=atos_csv_dict['Ato_Cessao'].iloc[0]

In [ ]:
open('cessao_exemplo.csv', 'w').write(ss.to_csv())

In [ ]:
len(ss.values), len(ss.index)

In [ ]:
df = pd.DataFrame(
    data=[ss.values], columns=ss.index
)
df.to_csv('cessao_exemplo.csv')

In [ ]:
df.columns

In [ ]:
import json
for v in df.iloc[0]:
    print(json.dumps(v), end=',\n')

In [ ]:
v.shape

In [ ]:
pd.read_csv('cessao_exemplo.csv')

In [ ]:
for k, v in atos_csv_dict.items():
    if 'labels_IOB' in v.columns:
        v.pop('labels_IOB')


In [ ]:
atos_csv_dict['Ato_Cessao'].columns

In [ ]:
mlflow.end_run()

In [ ]:
# %%time
# mlflow.set_registry_uri("")
mlflow.set_tracking_uri("localhost:5000")
training_ratio = 0.7
try:
    mlflow.end_run()
except:
    pass
run = mlflow.start_run()
print("RUN:", run.info.run_id)

params = dict(
    algorithm = 'l2sgd', 
    c2=1,
    max_iterations=10, 
    all_possible_transitions=True,
)

model = sklearn_crfsuite.CRF(
    **params,
    verbose=False
)

pipe = Pipeline([
    ('featurizer', FeatureTransformer()),
    ('model', model)
])

for key in atos_csv_dict.keys():
    sz = len(atos_csv_dict[key])
    limiar = math.floor(training_ratio*sz)
    print("------------------------------------------------------------------------------------")
    print("Ato:" + key)
    print("Tamanho: " + str(sz))
    df = atos_csv_dict[key].copy()
    
    if sz < 10:
        df_train, df_test = df.iloc[:limiar, :], df.iloc[limiar:, :]
        pipe.fit(
            df_train,
            IOB_Transformer().transform(df_train),
        );
        y_pred = pipe.predict(df_test)
        test_y = IOB_Transformer().transform(df_test)

        labels = list(pipe.classes_)
        labels.remove('O')

        f1 = metrics.flat_f1_score(test_y, y_pred, 
                              average='weighted', labels=labels)
    

        print(f1)
        print(metrics.flat_classification_report(
            test_y, y_pred, labels=labels, digits=3
        ))
        model = pipe

    else:
        X, y = df, IOB_Transformer().transform(df)
        res = cross_validate(
            pipe, X, y,
            cv = 3, return_estimator=True
        )
        model = res['estimator'][np.argmax(res['test_score'])]
        print(res)
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=f"model-{key}",
        registered_model_name=key,
    )
    mlflow.log_params(params)

In [ ]:
mlflow.end_run()